# Wprowadzenie do prawdopodobieństwa i statystyki
W tym notatniku pobawimy się niektórymi z koncepcji, które omawialiśmy wcześniej. Wiele koncepcji z prawdopodobieństwa i statystyki jest dobrze reprezentowanych w głównych bibliotekach do przetwarzania danych w Pythonie, takich jak `numpy` i `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Zmienne losowe i rozkłady
Zacznijmy od pobrania próbki 30 wartości z rozkładu jednostajnego od 0 do 9. Obliczymy również średnią i wariancję.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Aby wizualnie oszacować, ile różnych wartości znajduje się w próbce, możemy narysować **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Analiza rzeczywistych danych

Średnia i wariancja są bardzo ważne przy analizie danych ze świata rzeczywistego. Załadujmy dane o baseballistach z [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Używamy tutaj pakietu o nazwie [**Pandas**](https://pandas.pydata.org/) do analizy danych. O Pandas i pracy z danymi w Pythonie porozmawiamy więcej później w tym kursie.

Obliczmy średnie wartości dla wieku, wzrostu i wagi:


In [ ]:
df[['Age','Height','Weight']].mean()

Skoncentrujmy się teraz na wzroście i obliczmy odchylenie standardowe oraz wariancję:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Oprócz średniej warto spojrzeć na medianę i kwartyle. Można je zwizualizować za pomocą **wykresu pudełkowego**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Możemy także tworzyć wykresy pudełkowe podzbiorów naszego zbioru danych, na przykład pogrupowanych według roli zawodnika.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Uwaga**: Ten diagram sugeruje, że średnio wzrost pierwszych bazmanów jest wyższy niż wzrost drugich bazmanów. Później nauczymy się, jak bardziej formalnie przetestować tę hipotezę i jak wykazać, że nasze dane są statystycznie istotne, aby to udowodnić.  

Wiek, wzrost i waga to wszystkie ciągłe zmienne losowe. Jak myślisz, jaki mają rozkład? Dobrym sposobem, aby się tego dowiedzieć, jest wykreślenie histogramu wartości: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Rozkład normalny

Stwórzmy sztuczną próbkę wag, która będzie podążać za rozkładem normalnym o takim samym średnim i wariancji jak nasze rzeczywiste dane:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Ponieważ większość wartości w rzeczywistym życiu jest rozłożona normalnie, nie powinniśmy używać generatora liczb losowych o rozkładzie jednostajnym do generowania danych próbki. Oto co się stanie, jeśli spróbujemy wygenerować wagi z rozkładem jednostajnym (wygenerowanym przez `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Przedziały ufności

Obliczmy teraz przedziały ufności dla wag i wzrostów zawodników baseballu. Użyjemy kodu [z tej dyskusji na stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Testowanie hipotez

Przyjrzyjmy się różnym rolom w naszym zestawie danych graczy baseballowych:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Przetestujmy hipotezę, że gracze na pozycji pierwszej bazy są wyżsi niż gracze na pozycji drugiej bazy. Najprostszym sposobem jest sprawdzenie przedziałów ufności:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Widzimy, że przedziały nie nakładają się.

Statystycznie bardziej poprawnym sposobem na udowodnienie hipotezy jest użycie **testu t-Studenta**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dwie wartości zwracane przez funkcję `ttest_ind` to:
* wartość p można uznać za prawdopodobieństwo, że dwie rozkłady mają tę samą średnią. W naszym przypadku jest ona bardzo niska, co oznacza, że istnieją mocne dowody na to, że pierwszi bazowi są wyżsi.
* wartość t jest pośrednią wartością znormalizowanej różnicy średnich, która jest używana w teście t i porównywana z wartością progową dla określonego poziomu ufności.


## Symulacja rozkładu normalnego za pomocą twierdzenia granicznego centralnego

Generator pseudolosowy w Pythonie jest zaprojektowany tak, aby dawać rozkład jednostajny. Jeśli chcemy stworzyć generator dla rozkładu normalnego, możemy użyć twierdzenia granicznego centralnego. Aby uzyskać wartość o rozkładzie normalnym, po prostu obliczymy średnią z próbki wygenerowanej równomiernie.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelacja i Zła Korporacja Baseballowa

Korelacja pozwala nam znaleźć zależności między sekwencjami danych. W naszym przykładowym przykładzie załóżmy, że istnieje zła korporacja baseballowa, która płaci swoim graczom zgodnie z ich wzrostem - im wyższy zawodnik, tym więcej pieniędzy otrzymuje. Załóżmy, że jest podstawowa pensja w wysokości 1000$, oraz dodatkowa premia od 0 do 100$, w zależności od wzrostu. Weźmiemy prawdziwych graczy z MLB i obliczymy ich wyimaginowane wynagrodzenia:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Obliczmy teraz kowariancję i korelację tych sekwencji. `np.cov` da nam tzw. **macierz kowariancji**, która jest rozszerzeniem kowariancji na wiele zmiennych. Element $M_{ij}$ macierzy kowariancji $M$ jest kowariancją między zmiennymi wejściowymi $X_i$ i $X_j$, a wartości diagonalne $M_{ii}$ to wariancje $X_{i}$. Podobnie, `np.corrcoef` da nam **macierz korelacji**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelacja równa 1 oznacza, że istnieje silna **liniowa zależność** między dwiema zmiennymi. Możemy wizualnie zobaczyć liniową zależność, wykreślając jedną wartość względem drugiej:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Zobaczmy, co się stanie, jeśli relacja nie jest liniowa. Załóżmy, że nasza korporacja postanowiła ukryć oczywistą liniową zależność między wzrostem a wynagrodzeniem i wprowadziła do wzoru pewną nieliniowość, na przykład `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

W tym przypadku korelacja jest nieco mniejsza, ale nadal dość wysoka. Teraz, aby związek był jeszcze mniej oczywisty, możemy dodać trochę dodatkowego losowego elementu, dodając do wynagrodzenia jakąś losową zmienną. Zobaczmy, co się stanie:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Czy potrafisz zgadnąć, dlaczego kropki układają się w pionowe linie w ten sposób?

Zaobserwowaliśmy korelację między sztucznie zaprojektowaną koncepcją, taką jak wynagrodzenie, a obserwowaną zmienną *wzrost*. Zobaczmy także, czy dwie obserwowane zmienne, takie jak wzrost i waga, również korelują:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Niestety, nie otrzymaliśmy żadnych wyników – tylko kilka dziwnych wartości `nan`. Wynika to z faktu, że niektóre wartości w naszej serii są niezdefiniowane, reprezentowane jako `nan`, co powoduje, że wynik operacji również jest niezdefiniowany. Patrząc na macierz widzimy, że kolumna `Weight` jest problematyczna, ponieważ została obliczona autokorelacja pomiędzy wartościami `Height`.

> Ten przykład pokazuje, jak ważne jest **przygotowanie danych** i **czyszczenie**. Bez odpowiednich danych nie możemy nic obliczyć.

Użyjmy metody `fillna`, aby wypełnić brakujące wartości i obliczyć korelację:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Istnieje faktycznie korelacja, ale nie tak silna jak w naszym sztucznym przykładzie. Rzeczywiście, jeśli spojrzymy na wykres rozrzutu jednej wartości względem drugiej, związek byłby znacznie mniej oczywisty:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Wnioski

W tym notatniku nauczyliśmy się, jak wykonywać podstawowe operacje na danych, aby obliczać funkcje statystyczne. Wiemy teraz, jak używać solidnego aparatu matematycznego i statystycznego, aby udowodnić niektóre hipotezy oraz jak obliczać przedziały ufności dla dowolnych zmiennych, mając próbkę danych.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zrzeczenie się odpowiedzialności**:  
Niniejszy dokument został przetłumaczony przy użyciu automatycznej usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że dokładamy starań, aby tłumaczenie było jak najbardziej precyzyjne, prosimy mieć na uwadze, że automatyczne przekłady mogą zawierać błędy lub nieścisłości. Oryginalny dokument w języku źródłowym należy uważać za ostateczne i autorytatywne źródło. W przypadku informacji o krytycznym znaczeniu zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
